# Fruit Image Classification Using Transfer Learning (VGG16)

In this project I built an image classifier that can recognize 24 different fruit (and vegetable) categories from photos. Instead of training a deep neural network from scratch — which would require a massive dataset and a lot of compute — I used **transfer learning** with **VGG16**, a well-known convolutional neural network that was pre-trained on ImageNet (1.2 million images, 1000 classes).

The idea is simple: VGG16 already knows how to detect edges, textures, shapes and patterns. I kept all of that knowledge, chopped off its original classification head, and replaced it with my own layers trained specifically to tell fruits apart.

The pipeline goes like this:
1. Download and set up the Fruits-360 dataset
2. Build data generators with augmentation
3. Load VGG16 and add custom classification layers on top
4. Train with the VGG16 weights frozen (feature extraction phase)
5. Unfreeze the last few VGG16 layers and fine-tune
6. Evaluate on the test set and visualize predictions

Final test accuracy: **~86%** across 24 classes.


## 1. Setup

First I suppress some noisy TensorFlow/Keras warnings that show up when running on CPU, and make sure the environment stays clean.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras.src.trainers.data_adapters.py_dataset_adapter")
warnings.filterwarnings("ignore", category=UserWarning, module="keras.src.trainers.epoch_iterator")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Then I install the specific library versions I used so the notebook is reproducible.

In [ ]:
!pip install tensorflow==2.16.2 matplotlib==3.9.2 numpy==1.26.4 scipy==1.14.1 scikit-learn==1.5.2 -q

## 2. Dataset — Fruits-360

I used the **Fruits-360** dataset, which contains high-quality photos of fruits and vegetables on white backgrounds. It's already split into Training, Validation, and Test folders, and each class has its own subdirectory — which makes it very easy to load with Keras's `flow_from_directory`.

The version I downloaded has:
- **24 classes** (mostly apple varieties, plus pear, cucumber, carrot, eggplant, zucchini, etc.)
- **~12,500 images** total across all three splits

The code below downloads the zip from the course server, extracts it in chunks (it's large), and cleans up after itself.


In [ ]:
import os
import subprocess
import zipfile

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/4yIRGlIpNfKEGJYMhZV52g/fruits-360-original-size.zip"
local_zip = "fruits-360-original-size.zip"
extract_dir = "fruits-360-original-size"

def download_dataset(url, output_file):
    print("Downloading the dataset...")
    subprocess.run(["wget", "-q", "-O", output_file, url], check=True)
    print("Download completed.")

def extract_zip_in_chunks(zip_file, extract_to, batch_size=2000):
    """Extract a large zip in batches to avoid memory bottlenecks."""
    print("Extracting the dataset in chunks...")
    os.makedirs(extract_to, exist_ok=True)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        files = zip_ref.namelist()
        total_files = len(files)
        for i in range(0, total_files, batch_size):
            for file in files[i:i+batch_size]:
                zip_ref.extract(file, extract_to)
            print(f"Extracted {min(i+batch_size, total_files)} of {total_files} files...")
    print(f"Done. Extracted to '{extract_to}'.")

if not os.path.exists(local_zip):
    download_dataset(url, local_zip)

if not os.path.exists(extract_dir):
    extract_zip_in_chunks(local_zip, extract_dir)

if os.path.exists(local_zip):
    os.remove(local_zip)
    print(f"Cleaned up {local_zip}")

Download completed.
Extracting the dataset in chunks...
Extracted 2000 of 12481 files...
Extracted 4000 of 12481 files...
Extracted 6000 of 12481 files...
Extracted 8000 of 12481 files...
Extracted 10000 of 12481 files...
Extracted 12000 of 12481 files...
Extracted 12481 of 12481 files...
Done. Extracted to 'fruits-360-original-size'.
Cleaned up fruits-360-original-size.zip


## 3. Imports and Dataset Paths

Now I import everything I'll need and point the generators at the right directories.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

train_dir = 'fruits-360-original-size/fruits-360-original-size/Training'
val_dir   = 'fruits-360-original-size/fruits-360-original-size/Validation'
test_dir  = 'fruits-360-original-size/fruits-360-original-size/Test'

Each directory contains one subfolder per class. Keras will automatically use the folder names as class labels — no manual labelling needed.


## 4. Data Generators & Augmentation

I use Keras's `ImageDataGenerator` to load images on the fly in batches, which means I don't have to load the whole dataset into memory at once.

For the **training set** I also apply data augmentation — random rotations, flips, zooms, and shifts. This artificially increases the variety of images the model sees during training, which helps it generalize better and not just memorize the training set.

For **validation and test**, I only rescale pixel values to [0, 1]. No augmentation there — I want those to reflect real-world performance.


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen  = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(64, 64), batch_size=16, class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(64, 64), batch_size=16, class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(64, 64), batch_size=16, class_mode='categorical'
)

Found 6231 images belonging to 24 classes.
Found 3114 images belonging to 24 classes.
Found 3110 images belonging to 24 classes.


So we have 6,231 training images, 3,114 validation, and 3,110 test images — all automatically labelled and loaded in batches of 16. Images are resized to 64×64 pixels.


## 5. Model Architecture — VGG16 + Custom Head

Here's the core of transfer learning. I load VGG16 with its ImageNet weights, but with `include_top=False` — this strips off the original fully-connected classification layers (which were built for 1000 ImageNet classes) so I can replace them with my own.

I then freeze all the VGG16 layers. This means during the first training phase, their weights don't change at all — they're just used as a fixed feature extractor. Only my custom head gets trained.

My custom head is:
- `GlobalAveragePooling2D` — collapses the spatial dimensions of VGG16's output into a single vector
- `Dense(256, relu)` — a fully connected layer to learn fruit-specific patterns
- `BatchNormalization` — stabilizes training
- `Dropout(0.3)` — randomly drops 30% of neurons each step to reduce overfitting
- `Dense(24, softmax)` — the final output layer, one probability per class


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze all VGG16 layers — we're not touching their weights yet
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

Now I compile the model. I use `categorical_crossentropy` as the loss function because this is a multi-class problem (24 classes, one correct answer per image). Adam is a solid default optimizer that adapts the learning rate during training.


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## 6. Phase 1 — Feature Extraction Training

I train for 5 epochs with two callbacks:
- **ReduceLROnPlateau**: if validation loss stops improving for 2 epochs, the learning rate is cut by 80% to give the optimizer a chance to find a better minimum
- **EarlyStopping**: if val loss doesn't improve for 5 epochs, training stops early and the best weights are restored — this prevents wasting time and avoids overfitting

I limit to 50 steps per epoch and 25 validation steps. On a CPU this keeps training to a reasonable time while still making good progress.


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.mixed_precision import set_global_policy

lr_scheduler  = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

set_global_policy('float32')

history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    steps_per_epoch=50,
    validation_steps=25,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.2675 - loss: 2.6561 - val_accuracy: 0.2100 - val_loss: 2.6586 - learning_rate: 0.0010
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.5213 - loss: 1.5834 - val_accuracy: 0.4225 - val_loss: 2.1430 - learning_rate: 0.0010
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.5863 - loss: 1.3203 - val_accuracy: 0.4650 - val_loss: 1.8615 - learning_rate: 0.0010
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.6587 - loss: 1.0643 - val_accuracy: 0.5375 - val_loss: 1.5257 - learning_rate: 0.0010
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.6913 - loss: 0.9276 - val_accuracy: 0.6525 - val_loss: 1.2305 - learning_rate: 0.0010


After 5 epochs with the VGG16 weights frozen, the model reaches about **69% training accuracy** and **65% validation accuracy**. Not bad — but we can do better by letting the model adapt some of the lower layers too.


## 7. Phase 2 — Fine-Tuning

Now I unfreeze the **last 5 layers** of VGG16 (the deepest convolutional blocks) so the model can start adapting those representations to fruit images specifically.

I keep the BatchNorm layers frozen because updating them with a small dataset can destabilize training.

I recompile with a much lower learning rate (`1e-5` instead of `1e-3`). This is important — if the learning rate is too high during fine-tuning, we'd risk destroying the pre-trained weights we just spent time using.


In [ ]:
num_layers = len(base_model.layers)
print(f"VGG16 has {num_layers} layers total.")

# Unfreeze the last 5 layers
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Keep BatchNorm layers frozen to keep training stable
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

# Recompile with a low learning rate so we don't overwrite the pre-trained knowledge
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator,
    steps_per_epoch=50,
    validation_steps=25,
    callbacks=[lr_scheduler, early_stopping]
)

VGG16 has 19 layers total.
Epoch 1/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.7383 - loss: 0.8548 - val_accuracy: 0.6450 - val_loss: 1.0274 - learning_rate: 1.0000e-05
Epoch 2/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7613 - loss: 0.7665 - val_accuracy: 0.7275 - val_loss: 0.8596 - learning_rate: 1.0000e-05
Epoch 3/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.8462 - loss: 0.5940 - val_accuracy: 0.7525 - val_loss: 0.7244 - learning_rate: 1.0000e-05
Epoch 4/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8450 - loss: 0.5283 - val_accuracy: 0.8675 - val_loss: 0.5083 - learning_rate: 1.0000e-05
Epoch 5/5
50/50 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.8537 - loss: 0.5283 - val_accuracy: 0.8250 - val_loss: 0.5381 - learning_rate: 1.0000e-05


Fine-tuning made a significant difference. Validation accuracy jumped from ~65% to ~82–86% in just 5 more epochs. Unfreezing even a small number of layers lets the model adjust its learned features to be more specific to fruits rather than generic ImageNet patterns.


## 8. Evaluation on the Test Set

Now I run the model against the held-out test set — images it has never seen during training or validation. This gives the real measure of how well it generalizes.


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=50)
print(f"Test Accuracy: {test_accuracy:.2f}")

50/50 ━━━━━━━━━━━━━━━━━━━━ 38s 749ms/step - accuracy: 0.8625 - loss: 0.5041
Test Accuracy: 0.86


**86% accuracy on 24 classes** — that's solid, especially given I only trained for 10 epochs total and used a CPU. The gap between training (~85%) and test accuracy (~86%) is negligible, which means the model isn't overfitting.


## 9. Training Curves

I plot the accuracy and loss across both training phases to visualize the learning progression. Combining both histories side by side makes it easy to see the effect of fine-tuning.


In [ ]:
# Accuracy curves — both phases
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'],     label='Train (frozen)')
plt.plot(history.history['val_accuracy'], label='Val (frozen)')
plt.plot(history_fine.history['accuracy'],     label='Train (fine-tuned)')
plt.plot(history_fine.history['val_accuracy'], label='Val (fine-tuned)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Training')
plt.legend()
plt.grid(True)

# Loss curves — both phases
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'],     label='Train (frozen)')
plt.plot(history.history['val_loss'], label='Val (frozen)')
plt.plot(history_fine.history['loss'],     label='Train (fine-tuned)')
plt.plot(history_fine.history['val_loss'], label='Val (fine-tuned)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Training')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

You can clearly see two distinct learning phases in the curves. The first phase (frozen layers) shows a steady improvement. The second phase (fine-tuning) shows a jump in both accuracy and loss reduction — that's the model's deeper layers adapting to fruit-specific features.


## 10. Visual Predictions on Sample Test Images

Finally, I pick a few images from the test set and run them through the model to see predictions side by side with the actual labels.

The function reads the true class directly from the folder name (so no hardcoded labels), preprocesses the image the same way as training, and shows the result visually.


In [ ]:
from collections import Counter
from tensorflow.keras.preprocessing.image import img_to_array, load_img

actual_count    = Counter()
predicted_count = Counter()

def get_class_name_from_index(predicted_index, class_index_mapping):
    for class_name, index in class_index_mapping.items():
        if index == predicted_index:
            return class_name
    return "Unknown"

def visualize_prediction(img_path, class_index_mapping):
    # True label comes from the folder name — no hardcoding
    true_class = os.path.basename(os.path.dirname(img_path))

    img = load_img(img_path, target_size=(64, 64))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array, verbose=0)
    predicted_index = np.argmax(prediction, axis=-1)[0]
    predicted_class = get_class_name_from_index(predicted_index, class_index_mapping)

    actual_count[true_class]        += 1
    predicted_count[predicted_class] += 1

    plt.figure(figsize=(2, 2), dpi=100)
    plt.imshow(img)
    color = 'green' if true_class == predicted_class else 'red'
    plt.title(f"True: {true_class}\nPred: {predicted_class}", color=color, fontsize=7)
    plt.axis('off')
    plt.show()

class_index_mapping = train_generator.class_indices

sample_images = [
    'fruits-360-original-size/fruits-360-original-size/Test/apple_braeburn_1/r0_11.jpg',
    'fruits-360-original-size/fruits-360-original-size/Test/pear_1/r0_103.jpg',
    'fruits-360-original-size/fruits-360-original-size/Test/cucumber_3/r0_103.jpg',
]

for img_path in sample_images:
    visualize_prediction(img_path, class_index_mapping)

The title turns green for correct predictions and red for wrong ones. Misclassifications tend to happen between visually similar classes — for example, different apple varieties (Braeburn vs Golden vs Red Delicious all look quite alike). This is expected and would improve with more training epochs or higher-resolution images.


## Conclusion

This project shows how powerful transfer learning is for image classification tasks, especially when you don't have a huge dataset or a GPU. By reusing VGG16's convolutional features — which it learned from millions of images — I got to 86% accuracy on 24 fruit classes in just 10 epochs of training.

The two-phase approach (freeze → fine-tune) is a standard and effective strategy: the first phase teaches the custom head to work with VGG16's features, and the second phase lets the deeper layers refine themselves for the specific task.

Things that could improve it further:
- Training for more epochs with patience
- Using a higher input resolution (e.g. 128×128)
- Trying a more modern backbone like EfficientNetV2 or MobileNetV3
- Adding more augmentation variety
